# Introduction

**Resources**

* [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434) (2015) by Alec Radford, Luke Metz, Soumith Chintala
* [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167) (2015) by Sergey Ioffe, Christian Szegedy

Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Limit TensorFlow GPU memory usage


In [2]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config):
    pass  # init sessin with allow_growth

# MNIST Dataset

Load MNIST Dataset from Keras API. We only need train images, ignore labels and validation set.

In [3]:
(x_train_raw, _), (_, _) = tf.keras.datasets.mnist.load_data()

Convert to -1..1 range to mach tanh output from generator

In [4]:
x_train = (x_train_raw-127.5) / 127.5
x_train = x_train.reshape([len(x_train), -1])
print('x_train.shape:', x_train.shape)
print('x_train.min():', x_train.min())
print('x_train.max():', x_train.max())
print('x_train:\n', x_train)

x_train.shape: (60000, 784)
x_train.min(): -1.0
x_train.max(): 1.0
x_train:
 [[-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 ...
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]]


# DCGAN in Keras

In [ ]:
from tensorflow.keras.layers import InputLayer, Dense                # InputLayer for pretier names in TensorBoard

generator = tf.keras.Sequential(name='Generator')
generator.add(InputLayer(input_shape=(100,), name='GenInput'))       # random noise input
generator.add(Dense(units=128, activation='elu', name='GenHidden'))  # one hidden layer
generator.add(Dense(784, activation='tanh', name='GenOutput'))       # MNIST-like output
# generator.compile(...)                                             # no need
generator.summary()

In [7]:
n_random = 100
n_channels = 1

In [17]:
from tensorflow.keras.layers import InputLayer, Dense, Reshape, UpSampling2D, Conv2D, BatchNormalization, Activation

generator = tf.keras.Sequential()
generator.add(InputLayer(input_shape=(n_random,)))
generator.add(Dense(7*7*128, activation="relu"))
generator.add(Reshape([7,7,128]))
generator.add(BatchNormalization())

# generator.add(UpSampling2D())
# generator.add(Conv2D(128, kernel_size=3, padding="same"))
# generator.add(BatchNormalization(momentum=0.8))

# generator.add(Activation("relu"))
# generator.add(UpSampling2D())
# generator.add(Conv2D(64, kernel_size=3, padding="same"))
# generator.add(BatchNormalization(momentum=0.8))
# generator.add(Activation("relu"))
# generator.add(Conv2D(n_channels, kernel_size=3, padding="same"))
# generator.add(Activation("tanh"))

generator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [12]:
generator = build_generator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6272)              633472    
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 128)       512       
_________________________________________________________________
activation (Activation)      (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 128)       0         
__________

NameError: name 'Input' is not defined